# Preprocessing

In [1]:
import pandas as pd

## Load the Data

Load the downloaded data:

In [2]:
DATA_FILEPATH = "../../data/stocks.csv"
stocks = pd.read_csv(DATA_FILEPATH, index_col=0, parse_dates=True).to_period("D")
stocks

,Company,Sector,Open,High,Low,Close,Volume
Date,,,,,,,
2017-11-02,AAPL,Technology,41.650002,42.125000,41.320000,42.027500,165573600
2017-11-03,AAPL,Technology,43.500000,43.564999,42.779999,43.125000,237594400
2017-11-06,AAPL,Technology,43.092499,43.747501,42.930000,43.562500,140105200
2017-11-07,AAPL,Technology,43.477501,43.812500,43.400002,43.702499,97446000
2017-11-08,AAPL,Technology,43.665001,44.060001,43.582500,44.060001,97638000
...,...,...,...,...,...,...,...
2022-10-26,COP,Energy,124.720001,128.179993,124.580002,126.570000,8139100
2022-10-27,COP,Energy,127.699997,129.449997,126.239998,126.639999,8948500
2022-10-28,COP,Energy,128.500000,128.990005,124.010002,127.169998,7293200


## Resampling

Note that the data at weekends are missing since markets are closed at weekends. For example, there is no data on 2017/11/04 and 2017/11/05 for Apple company.

To fix this, we can resample the data in each day. We can do so using `DataFrame`'s `resample` method.

Moreover, before resampling, we need to first group the data frame by the `"Company"` and `"Sector"` fields.

In [100]:
resampled_stocks: pd.DataFrame = stocks.groupby(["Company", "Sector"]).resample("D").mean()
resampled_stocks

Open        High         Low       Close  \
Company Sector     Date                                                         
AAPL    Technology 2017-11-02   41.650002   42.125000   41.320000   42.027500   
                   2017-11-03   43.500000   43.564999   42.779999   43.125000   
                   2017-11-04         NaN         NaN         NaN         NaN   
                   2017-11-05         NaN         NaN         NaN         NaN   
                   2017-11-06   43.092499   43.747501   42.930000   43.562500   
...                                   ...         ...         ...         ...   
XOM     Energy     2022-10-28  109.459999  111.209999  108.120003  110.699997   
                   2022-10-29         NaN         NaN         NaN         NaN   
                   2022-10-30         NaN         NaN         NaN         NaN   
                   2022-10-31  109.500000  112.910004  109.440002  110.809998   
                   2022-11-01  112.370003  112.570000  110.650002  111.910004   

                                    Volume  
Company Sector     Date                     
AAPL    Technology 2017-11-02  165573600.0  
                   2017-11-03  237594400.0  
                   2017-11-04          NaN  
                   2017-11-05          NaN  
                   2017-11-06  140105200.0  
...                                    ...  
XOM     Energy     2022-10-28   28169400.0  
                   2022-10-29          NaN  
                   2022-10-30          NaN  
                   2022-10-31   26401800.0  
                   2022-11-01   22205500.0  

[91992 rows x 5 columns]

The argument `"D"` (stands for *day*) passed to the `resample` method means that we want to resample the data in each day. In addition, note that we also called the `mean` method in the end. Then the value of each column will be replaced with the mean value over the day. If there is no data in that day, the mean value is `NaN`.

## Fill in Missing Data

Now, we need to fill in the missing data. A reasonable method is linear interpolation. Suppose that the value on date $1$ is $x_1$ and the value on date $n$ is $x_n$, and the values from date $2$ to date $n-1$ are all missing. Consider the time plot of this time series. Then the missing values are imputed with the values falling on the line segment from $(\text{date} \; 1, x_1)$ to $(\text{date}\; n, x_n)$. `DataFrame`'s `interpolate` can achieve this easily.

In [101]:
filled_stocks = resampled_stocks.interpolate("linear")
filled_stocks

Open        High         Low       Close  \
Company Sector     Date                                                         
AAPL    Technology 2017-11-02   41.650002   42.125000   41.320000   42.027500   
                   2017-11-03   43.500000   43.564999   42.779999   43.125000   
                   2017-11-04   43.364166   43.625833   42.829999   43.270833   
                   2017-11-05   43.228333   43.686667   42.880000   43.416667   
                   2017-11-06   43.092499   43.747501   42.930000   43.562500   
...                                   ...         ...         ...         ...   
XOM     Energy     2022-10-28  109.459999  111.209999  108.120003  110.699997   
                   2022-10-29  109.473333  111.776667  108.560003  110.736664   
                   2022-10-30  109.486666  112.343335  109.000003  110.773331   
                   2022-10-31  109.500000  112.910004  109.440002  110.809998   
                   2022-11-01  112.370003  112.570000  110.650002  111.910004   

                                    Volume  
Company Sector     Date                     
AAPL    Technology 2017-11-02  165573600.0  
                   2017-11-03  237594400.0  
                   2017-11-04  205098000.0  
                   2017-11-05  172601600.0  
                   2017-11-06  140105200.0  
...                                    ...  
XOM     Energy     2022-10-28   28169400.0  
                   2022-10-29   27580200.0  
                   2022-10-30   26991000.0  
                   2022-10-31   26401800.0  
                   2022-11-01   22205500.0  

[91992 rows x 5 columns]

## Save the Processed Data Frame

Through our project, we will continue to work on this processed data frame. To easily load this data frame, we save it as a pickle (`.pkl`) file.

In [102]:
import os

DATA_DIR = "../../data/"
filepath = os.path.join(DATA_DIR, "stocks.pkl")
filled_stocks.to_pickle(filepath)

Read the pickle file to see whether the data frame is as desired:

In [104]:
saved_stocks = pd.read_pickle(filepath)
saved_stocks

Open        High         Low       Close  \
Company Sector     Date                                                         
AAPL    Technology 2017-11-02   41.650002   42.125000   41.320000   42.027500   
                   2017-11-03   43.500000   43.564999   42.779999   43.125000   
                   2017-11-04   43.364166   43.625833   42.829999   43.270833   
                   2017-11-05   43.228333   43.686667   42.880000   43.416667   
                   2017-11-06   43.092499   43.747501   42.930000   43.562500   
...                                   ...         ...         ...         ...   
XOM     Energy     2022-10-28  109.459999  111.209999  108.120003  110.699997   
                   2022-10-29  109.473333  111.776667  108.560003  110.736664   
                   2022-10-30  109.486666  112.343335  109.000003  110.773331   
                   2022-10-31  109.500000  112.910004  109.440002  110.809998   
                   2022-11-01  112.370003  112.570000  110.650002  111.910004   

                                    Volume  
Company Sector     Date                     
AAPL    Technology 2017-11-02  165573600.0  
                   2017-11-03  237594400.0  
                   2017-11-04  205098000.0  
                   2017-11-05  172601600.0  
                   2017-11-06  140105200.0  
...                                    ...  
XOM     Energy     2022-10-28   28169400.0  
                   2022-10-29   27580200.0  
                   2022-10-30   26991000.0  
                   2022-10-31   26401800.0  
                   2022-11-01   22205500.0  

[91992 rows x 5 columns]

In [4]:
stocks = pd.read_csv("../../data/stocks.csv", index_col=0, parse_dates=True)
stocks

,Company,Sector,Open,High,Low,Close,Volume
Date,,,,,,,
2017-11-02,AAPL,Technology,41.650002,42.125000,41.320000,42.027500,165573600
2017-11-03,AAPL,Technology,43.500000,43.564999,42.779999,43.125000,237594400
2017-11-06,AAPL,Technology,43.092499,43.747501,42.930000,43.562500,140105200
2017-11-07,AAPL,Technology,43.477501,43.812500,43.400002,43.702499,97446000
2017-11-08,AAPL,Technology,43.665001,44.060001,43.582500,44.060001,97638000
...,...,...,...,...,...,...,...
2022-10-26,COP,Energy,124.720001,128.179993,124.580002,126.570000,8139100
2022-10-27,COP,Energy,127.699997,129.449997,126.239998,126.639999,8948500
2022-10-28,COP,Energy,128.500000,128.990005,124.010002,127.169998,7293200


In [9]:
demo = stocks.query("Company == 'AAPL'").drop(columns=["Company", "Sector"])
demo

,Open,High,Low,Close,Volume
Date,,,,,
2017-11-02,41.650002,42.125000,41.320000,42.027500,165573600
2017-11-03,43.500000,43.564999,42.779999,43.125000,237594400
2017-11-06,43.092499,43.747501,42.930000,43.562500,140105200
2017-11-07,43.477501,43.812500,43.400002,43.702499,97446000
2017-11-08,43.665001,44.060001,43.582500,44.060001,97638000
...,...,...,...,...,...
2022-10-26,150.960007,151.990005,148.039993,149.350006,88194300
2022-10-27,148.070007,149.050003,144.130005,144.800003,109180200
2022-10-28,148.199997,157.500000,147.820007,155.740005,164762400


In [10]:
demo.to_pickle("../../data/demo.pkl")